# DO THIS :  

### *** Names: [Insert Your Names Here;      Optional: indicate preferred pronouns]***

# AND IN THE FILENAME

##### Problem Set 13             
## PHYS 105A   Spring 2019

## Contents

More recursion, matplotlib, and fractals.

** There are 6 exercises that must be completed. **

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In all of today's problem set, we will be representing points as numpy arrays of length 2

Here is a unit vector pointing in the y direction

In [2]:
pt = np.array([0,1])

We will need to rotate these vectors by some angle, so we need to create a function which does this. To rotate a vector
$v = (x,y)$ by angle $\theta$ about the origin, the components of the rotated vector $v' = (x',y')$ are

$$
\begin{split}
x' &= x \cos{\theta}- y \sin{\theta}\\
y' &= x \sin{\theta}+ y \cos{\theta}
\end{split}
$$


We will use the 2D *rotation matrix* to express this as a matrix multiplication. The rotation matrix is

$$ 
R(\theta) = \left[ \begin{array}[ll] d
\cos{\theta} & -\sin{\theta} \\
\sin{\theta} & \cos{\theta}
\end{array} \right]
$$

With $R(\theta)$, the rotation operation becomes

$$ v' = R(\theta) v $$

In `numpy`, matrix multiplication is done with the dot() function, so our rotate function becomes

In [3]:
def rotate(vector, theta):
    """
    rotate point vector by theta about the origin
    """
    R = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    return np.dot(R, vector)

Let's try rotating the point above by 90 degrees clockwise. All angles in Python are in radians, so we need to use
$\theta = -\pi/2$

In [4]:
rotate(pt, -np.pi/2)

array([1.000000e+00, 6.123234e-17])

Just as we expected, we get a unit vector pointing in the x-direction.

#### Exercise 1

Using the rotate function, make a plot which places a dot at the positions of the numbers on a clock face (no numbers, just a dot will do).

To do so:
* create a unit vector pointing up.
* create a figure and axes with  

      fig, ax = plt.subplots()
      ax.set_aspect(1)

  (the set_aspect(1) keeps the plot square so your points will look as if they are in a circle)
* in a for loop, keep rotating the vector by 1/12 of a full circle and ploting a dot at the coordinates of the rotated vector

In most of the plotting we will do today, we will be creating a *path* -- a list of points -- and then plotting the curve represented by that path. Our list of points will be a Python list whose elements are numpy arrays of length 2. For example, we can explicitly write such a path consisting of three points as

    path = [ np.array([ 0, 0]), np.array([ 1, 1]), np.array([ 2, 0]) ]
    
We can then plot the path using a simple function like:

In [ ]:
def plotPath(path, c):
    x = []
    y = []
    for i in range(len(path)):
        x.append(path[i][0])
        y.append(path[i][1])
    ax.plot(x,y, color=c)
    
fig, ax = plt.subplots(figsize=(8,8))

p = [ np.array([ 0, 0]), np.array([ 1, 1]), np.array([ 2, 0]) ]

plotPath(p, 'g')

Note that while our path had three points, it didn't plot a triangle since the path was not *closed*. To close it,
we need to add a point at the end equal to the first point in the path:

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

p.append(p[0])
plotPath(p, 'r')

#### Exercise 2

Following on from the clock exercise above, create a funtion called polygon($n$) which creates a path describing a closed polygon with $n$ vertices, of unit radius.

Use your polygon function to create a path with 7 vertices, and then use the plotPath function to plot it. Don't forget to make a new fig,ax pair and ax.set_aspect(1)

Hint: The easiest way to create a closed path is to iterate one extra time so that the unit vector rotates back to the beginning position.

Armed with this plotting technology, we can write some code to make a fractal known as the Koch Snowflake. This is one of a class of fractals which are produced by recursively replacing each segment of a path with a more complex path.

Here is a function which illustrates how this might be done. The function takes a path of length 2 -- a line -- and returns a path consisting of 4 points:
* the original starting point
* a point 1/3 along, in the direction of the original segment
* a point 1/3 along, rotated by 60 degrees w/r to the original direction
* a point 1/3 along, rotated by -60 degrees w/r to the original direction


In [ ]:
def replaceSegment(segment):
    """
    Given two points forming a line segment, replace them with four points
    """

    angle = 60 * np.pi/180
    dl = segment[1]-segment[0]
    ratio = 1/3 

    p0 =  segment[0]                     # first point is the same
    p1 =  p0 + ratio*dl                  # second point is 1/3 of the way in the original direction
    p2 =  p1 + ratio*rotate(dl, -angle)  # third point is rotated outward by 60 degrees
    p3 =  p2 + ratio*rotate(dl,  angle)  # fourth point is rotated inward by 60 degrees (120 from previous)
                                         # last point is first point of next segment, so we don't need to
                                         # add it here
    return  [p0,p1,p2,p3]

Let's try this out on a simple unit vector in the x-direction

In [ ]:
path = [ np.array([0,0]), np.array([1,0]) ]
newpath = replaceSegment(path)


fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect(1)
plotPath(path, 'r')
plotPath(newpath, 'g')


We didn't add the end point to the path, since if the path contained more segments, the first point of the next segment is the last point of the previous segment. Thus, let's make a closed triangle path and try out our function

In [ ]:
path = polygon(3)

newpath = []
for i in range(len(path)-1):                      # there are 
    newpath += replaceSegment(path[i:i+2])
newpath.append(path[0])                           # close the path
    
fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect(1)
plotPath(path,'r')
plotPath(newpath,'b')

We can make our path-replacement code into a function which applies the replaceSegment function to all segments of a path
given as an argument:

In [ ]:
def replacePath(path):
    newpath = []
    for i in range(len(path)-1):                      # there are 
        newpath += replaceSegment(path[i:i+2])
    newpath.append(path[0])                           # close the path
    return newpath

fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect(1)

path = polygon(3)
plotPath(path,'r')

newpath = replacePath(path)
plotPath(newpath,'b')

#### Exercise 3

Try applying replacePath three times to our original path, and plot the result

Had you applied replacePath an infinite number of times, you would have created the fractal curve known as a Koch snowflake. Each application of the path replacement multiplies the number of sides by 4 and divides the length of the sides by 3, so the total perimiter length after $n$ applications is

$$ P_n = 3 \left(\frac{4}{3}\right)^n $$

We have an example of an closed curve with infinitely long perimiter enclosing a finite area!

#### Exercise 4

Write a recursive function to create an approximation to a Koch snowflake. Your function should take an original path as an argument (our triangle of unit side) and the number of applications $n$ to perform, and return a new path containing the snowflake path after $n$ iterations.

Note: You will need a base case to keep from recursing infinitely many times! You can stop the recursion after $n$ iterations with the following idea:

    def recurse(n, ...):  # ... means other arguments!
        if n == 0:
            return
            
        recurse(n-1, ...)
        
You call recurse the first time with the number of iterations. Every time the function calls itself, it decrements its argument, and when it is called with n=0, the recursion stops.

Try using $n=5$ and plot the resulting path.

Let's try another fractal geometric construction, the Sierpinski Gasket. This starts with a triangle, and then subtracts from it the inscribed triangle formed from the midpoints of the original triangles sides. The process is repeated with each of the four new triangles, and so on.

We'll have to draw a lot of triangles. If we draw them as closed paths, there will be a lot of closed paths to draw, and matplotlib slows down considerably when there are a lot of individual plot objects in the plot.

This is where *patches* come in. They are a more-efficient way of drawing closed curves, especially useful when drawing large numbers of such curves.

To see how this works, lets write a function to add a triangle to a list of patches. 

* First we'll need to import the collections module from matplotlib.
* Next we define out addTriangle function which takes a 3-point path describing a triangle. The Polygon patch will close the path for us. The function stores patch and color information in a list-of-lists.
* The we'll create a bunch of random 3-point paths and add them to our patch list.
* we can then give our list to matplotlib's PatchCollection function, and add the object it returns to our axes.
* Calling ax.autoscale() at the end will set the plot limits automagically.


In [ ]:
from matplotlib import collections as mc

def addTriangle(path, color, patches):
    poly = plt.Polygon(path, closed=True)     # create patch & automatically close it -- only need 3 points here
    patches[0].append(poly)               # append to list of patches
    patches[1].append(color)              # append color to list of colors

def plotPatches(patches):
    c = mc.PatchCollection(patches[0], facecolors=patches[1], edgecolors='k', linewidths=0.5)
    ax.add_collection(c)

    
patches = [ [], [] ]            # empty 2D list
colors = ['m','y','c','orange','g','r','b']

for i in range(10):
    path = [ np.random.random(2), np.random.random(2), np.random.random(2) ] # create a random triangle
    addTriangle(path,colors[i%7],patches)
    
fig, ax = plt.subplots(figsize=(8,8))
plotPatches(patches)

ax.autoscale()

On to the Gasket! The basic iteration step is to take a triangle, expressed as a path with 3 points,
draw it, and then make three new triangles, and draw them. The new triangles each have one of the original points and the two nearest midpoints as vertices:

In [ ]:
path = polygon(3)             # original triangle

patches = [ [], [] ]
addTriangle(path, 'b', patches)

m01 = 0.5*(path[0]+path[1])   # make midpoints of edges
m02 = 0.5*(path[0]+path[2])
m12 = 0.5*(path[1]+path[2])

newtriangle = [ path[0], m01, m02 ]     # first new triangle
addTriangle(newtriangle, 'g', patches)

newtriangle = [ path[1], m01, m12 ]     # second new triangle
addTriangle(newtriangle, 'g', patches)

newtriangle = [ path[2], m02, m12 ]     # third new triangle
addTriangle(newtriangle, 'g', patches)

fig, ax = plt.subplots(figsize=(8,8))
plotPatches(patches)

ax.autoscale()

#### Exercise 5

We now know enough to write a recursive Sierpinski Gasket funtion:

* Your function should take as arguments the desired number of iterations, the initial isosceles triangle, and an empty patches 2D list.
* The function should first add the triangle (given as an argument) to the patches list, just as in the code above.
* Then, if n==0, the function should just return.
* Otherwise, you make the 3 triangles, and instead of adding them to the patch list, you recusively call your function
  with n-1, the new triangle, and the patch list as arguments.
  
You can color the triangles using the iteration number $n$ as in the random triangle example above. Don't forget to use

    c = colors[n%7]
    
so you don't run off the end of the list for large $n$.

The gasket is made by recursive cutting out smaller and smaller parts of the original triangle, colored purple if you use the same color list defined above. The result is a construction which seems to fill the original triangle, but has zero area! Every iteration, the area is $3/4$ the previous remaining area. After an infinite number of recursions, there is nothing left.

One can make a Sierpinski gasket another way, by recursive drawing tridents.

Here is a bit of code which implements this method. Instead of working on triangles, it works on a list of line segments.
At each iteration, three lines are added at the end of each line, one heading to the left by 120 degrees, one straight ahead, and one to the right by 120 degrees, all of half the previous line's length.

In [ ]:
def recurse(n, linelist, a):
    """
    For each line in linelist, add 3 more lines half the length at angles -a,0,a
    """

    shrink = 0.5             # decrease in line length per iteration
    angles = [-a, 0, a]      # angles of the new lines, relative to current line

    if n == 0:
        return linelist

    newlines = []
    for line in linelist:
        p0 = line[0]         # starting point of original line segment
        p1 = line[1]         # ending point of original line segment
        segment = p1-p0
        for j in range(3):
            newlines.append([ p1, p1 + shrink*rotate(segment, angles[j]) ])

    result = recurse(n-1, newlines, a)
    return linelist + result

angle = 120 * np.pi/180
linelist = []
for i in range(-1,2):
    linelist.append([np.zeros(2),rotate(np.array([0,1]),i*angle)])

linelist = recurse(8, linelist, angle)

fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect(1)
ax.axis('off')

lc = mc.LineCollection(linelist, colors='k', linewidths=1)
ax.add_collection(lc)
ax.autoscale()

#### Exercise 6

It is interesting to observe what happens when one changes the parameters of the recursion. These are the angle and the factor by which the length of the additional lines shrink.

Copy all of the code in the example above to the following cell.  Decrease the angle to 20 degrees and increase the shrink factor to 0.75 This is the basis for the way many plants are made, as algorithms, for scenes in computer graphics (in the movie *Avatar*, for example).

You can take one step to making a (slightly) more realistic-looking plant (at least in 2D!), by adding random perturbation to the angles and shrink factors which vary during the iteration. A good place to start is by
defining a function to give a random number on $[-s,s]$

    def ran(s):
        return s*(2*np.random.random()-1)
        
You can then add ran(s) to all of the angles, and multiply the shrink factors (as well as the original unit vector) by (1+ran(s))

By varying $s$ and the number of iterations, you can begin to see how a wide variety of botanical forms can be created. 

If you have time, you could also vary the thickness of the lines as a function of recursion depth. This would mean
making linelist a 2D array as we did with the triangle patches and colors in the examples above.